In [ ]:
# 데이터 로드
from google.colab import drive

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

import pandas as pd

# 데이터셋 분할
from sklearn.model_selection import train_test_split

# 시퀀스 신경망, 은닉층, 드롭아웃 신경망
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 정규화 패키지
from sklearn.preprocessing import MinMaxScaler


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### 함수 정의 구간
# 데이터 업로드 _utf
def upload_data_utf(file_name):
  rtn = pd.read_csv(file_name, encoding = 'utf-8')
  rtn = pd.DataFrame(rtn)

  return rtn

def upload_data_cp(file_name):
  rtn = pd.read_csv(file_name, encoding = 'cp949', delimiter = '\t')
  rtn = pd.DataFrame(rtn)

  return rtn

### 데이터 정규화
  # Min-Max Scaling
  # 0에서 1사이로 정규화
def scaling(data, loctaion):

  # 스케일 대상 column
  scaleColumn = ['건축년도', '전용면적'] + location
  rtn = data

  # 데이터 정규화
  scaler = MinMaxScaler()
  rtn[scaleColumn] = scaler.fit_transform(rtn[scaleColumn])

  return rtn

def create_model(train_data, dropout_rate):
  # 다층 퍼셉트론(MLP) 모델 생성
  model = Sequential()
  model.add(Dense(128, activation='relu', input_shape=(train_data.shape[1],)))  # 첫 번째 은닉층
  model.add(Dropout(dropout_rate))
  model.add(Dense(64, activation='relu'))  # 두 번째 은닉층
  model.add(Dense(1))  # 출력층

  return model

def div_amount(amount, season):
    adjusted_amount = amount / season * 100

    return adjusted_amount

In [ ]:
# 안양 XDATA 파일 업로드
x_data = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/AnyangData/XDATA.csv")
y_data = x_data[['거래금액', '거래일자']]

In [ ]:
# 입지 정보 칼럼명
location = ['약국', '의원', '병원', '대병원', '대형상권','공원', '도서관', '유치원', '초등학교', '중학교', '고등학교', '학원', '지하철']

y_data = x_data[['거래금액']]
y_data['거래금액'] = y_data['거래금액'].str.replace(',', '').astype(int
                                                            )
x_data = x_data[['아파트', '건축년도', '전용면적'] + location]

In [ ]:
# 데이터 정규화
x_data = scaling(x_data, location)
x_data = x_data[['건축년도', '전용면적'] + location]

# 훈련 세트와 테스트 세트로 데이터 나누기
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
model = create_model(x_train, 0.2)
model.compile(optimizer = 'adam', loss = 'mse')

# 모델 학습
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/100
77/77 [==============================] - 1s 5ms/step - loss: 4008197888.0000 - val_loss: 4158449920.0000
Epoch 2/100
77/77 [==============================] - 0s 3ms/step - loss: 3982487296.0000 - val_loss: 4099478016.0000
Epoch 3/100
77/77 [==============================] - 0s 3ms/step - loss: 3870225152.0000 - val_loss: 3913584128.0000
Epoch 4/100
77/77 [==============================] - 0s 4ms/step - loss: 3604473856.0000 - val_loss: 3539496192.0000
Epoch 5/100
77/77 [==============================] - 0s 4ms/step - loss: 3146527488.0000 - val_loss: 2968565504.0000
Epoch 6/100
77/77 [==============================] - 0s 3ms/step - loss: 2531469056.0000 - val_loss: 2291833856.0000
Epoch 7/100
77/77 [==============================] - 0s 3ms/step - loss: 1878962944.0000 - val_loss: 1654502528.0000
Epoch 8/100
77/77 [==============================] - 0s 4ms/step - loss: 1344053376.0000 - val_loss: 1216077952.0000
Epoch 9/100
77/77 [==============================] - 0s 4ms/step

In [ ]:
predictions = model.predict(x_test)
print(predictions)

20/20 [==============================] - 0s 3ms/step


In [ ]:
mse = mean_squared_error(y_test, predictions)
print(f"평균 제곱 오차 (MSE): {mse}")

r2 = r2_score(y_test, predictions)
print(f"R square : {r2}")

평균 제곱 오차 (MSE): 263064555.84783497
R square : 0.475288111863984
